In [1]:
from utils import tokenize, load_curpus
import numpy as np

#### 加载数据

In [2]:
import pandas as pd
train_data = load_curpus("weibo2018/train.txt")
test_data = load_curpus("weibo2018/test.txt")
train_df = pd.DataFrame(train_data, columns=["content", "sentiment"])
test_df = pd.DataFrame(test_data, columns=["content", "sentiment"])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.449 seconds.
Prefix dict has been built succesfully.


In [3]:
train_df.head()

,content,sentiment
0,"[书中, 自有, 黄金屋, 书中, 自有, 颜如玉, 沿着, 岁月, 的, 长河, 跋涉, ...",1
1,"[这是, 英超, 被, 黑, 的, 最惨, 的, 一次, [二哈], [二哈], 十几年来,...",0
2,"[中国, 远洋, 海运, 集团, 副总经理, 俞曾, 港, 月, 日, 在, 上, 表示, ...",1
3,"[看, 流星花园, 其实, 也, 还好, 啦, 现在, 的, 观念, 以及, 时尚, 眼光,...",1
4,"[汉武帝, 的, 罪己, 诏, 的, 真实性, 尽管, 存在, 着, 争议, 然而, 轮台,...",1


加载停用词

In [4]:
stopwords = []
with open("stopwords.txt", "r", encoding="utf8") as f:
    for w in f:
        stopwords.append(w.strip())

TfIdf

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
data_str = [" ".join(content) for content, sentiment in train_data] + \
            [" ".join(content) for content, sentiment in test_data]
tfidf = TfidfVectorizer(token_pattern='\[?\w+\]?', stop_words=stopwords)
tfidf_fit = tfidf.fit_transform(data_str)

加载之前训练好的FastText模型

In [6]:
from gensim.models import FastText
model = FastText.load("model/model_100.txt")

In [7]:
max_length = 128

In [8]:
X_train, train_length, y_train = [], [], []
for content, sentiment in train_data:
    X, y = [], sentiment
    for w in content[:max_length]:
        if w in model:
            X.append(np.expand_dims(model[w], 0))
    if X:
        length = len(X)
        X = X + [np.zeros_like(X[0])] * (max_length - length)
        X = np.concatenate(X)
        X = np.expand_dims(X, 0)
        X_train.append(X)
        train_length.append(length)
        y_train.append(y)

/home/albert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/home/albert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [9]:
X_test, test_length, y_test = [], [], []
for content, sentiment in test_data:
    X, y = [], sentiment
    for w in content[:max_length]:
        if w in model:
            X.append(np.expand_dims(model[w], 0))
    if X:
        length = len(X)
        X = X + [np.zeros_like(X[0])] * (max_length - length)
        X = np.concatenate(X)
        X = np.expand_dims(X, 0)
        X_test.append(X)
        test_length.append(length)
        y_test.append(y)

/home/albert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/home/albert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


### LSTM

In [11]:
import tensorflow as tf
from tensorflow.contrib import rnn
batch_size = 512
lr = 1e-3
hidden_size = 100

X = tf.placeholder(shape=(batch_size, max_length, 100), dtype=tf.float32, name="X")
L = tf.placeholder(shape=(batch_size), dtype=np.int32, name="L")
y = tf.placeholder(shape=(batch_size, 1), dtype=np.float32, name="y")
dropout = tf.placeholder(shape=(), dtype=np.float32, name="dropout")
with tf.variable_scope("lstm", reuse=tf.AUTO_REUSE):
    def lstm_cell(hidden_size, cell_id=0):
        # LSTM细胞生成器
        cell = rnn.LSTMCell(hidden_size, reuse=tf.AUTO_REUSE, name='cell%d' % cell_id)
        cell = rnn.DropoutWrapper(cell, output_keep_prob=dropout)
        return cell
    
    cell = rnn.MultiRNNCell([lstm_cell(hidden_size, 0),
                              lstm_cell(hidden_size, 1)], state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    cell_output, cell_state = tf.nn.dynamic_rnn(cell, X, 
                                                sequence_length=L, 
                                                initial_state=initial_state, 
                                                dtype=tf.float32)
    W1 = tf.get_variable("W1", shape=(hidden_size, 50))
    b1 = tf.get_variable("b1", shape=(50,))
    W2 = tf.get_variable("W2", shape=(50, 1))
    b2 = tf.get_variable("b2", shape=(1,))
    fcn1 = tf.nn.xw_plus_b(cell_state[1][1], W1, b1)
    logists = tf.nn.xw_plus_b(fcn1, W2, b2)
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logists, labels=y))
    op = tf.train.AdamOptimizer(lr).minimize(loss)

/home/albert/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
config = tf.ConfigProto(gpu_options=gpu_options)
sess = tf.Session(config=config)

In [13]:
total_step = 1001
step = 0
cursor = 0
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=1)
while step < total_step:
    _X, _L, _y = X_train[cursor: cursor + batch_size], train_length[cursor: cursor + batch_size], y_train[cursor: cursor + batch_size]
    cursor += batch_size
    if len(_X) < batch_size:
        cursor = batch_size - len(_X)
        _X += X_train[: cursor]
        _L += train_length[: cursor]
        _y += y_train[: cursor]
    _X = np.concatenate(_X)
    _L = np.reshape(np.array(_L, dtype=np.int32), (-1))
    _y = np.reshape(np.array(_y, dtype=np.float32), (batch_size, 1))
    _, l = sess.run([op, loss], feed_dict={X: _X, L:_L, y: _y, dropout:.75})
    if step % 100 == 0:
        print("step:", step, " loss:", l)
        saver.save(sess,'model/lstm/model', global_step=step)
    step += 1

step: 0  loss: 0.8914259
step: 100  loss: 0.3929717
step: 200  loss: 0.34186012
step: 300  loss: 0.11103628
step: 400  loss: 0.10736647
step: 500  loss: 0.004171503
step: 600  loss: 0.002346907
step: 700  loss: 0.0007136409
step: 800  loss: 0.0005215733
step: 900  loss: 0.00015302733
step: 1000  loss: 0.00046443113


In [14]:
_X = np.concatenate(X_test + [np.zeros_like(X_test[0])] * (batch_size - len(X_test)))
_L = np.array(test_length + [0] * (batch_size - len(test_length)))

In [15]:
result = sess.run(tf.nn.sigmoid(logists), feed_dict={X: _X, L: _L, dropout:1.})
prediction = []
for i in result[:len(X_test)]:
    if i > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)

In [16]:
from sklearn import metrics
print(metrics.classification_report(y_test, prediction))
print("准确率:", metrics.accuracy_score(y_test, prediction))

             precision    recall  f1-score   support

          0       0.74      0.82      0.78       155
          1       0.91      0.87      0.89       345

avg / total       0.86      0.85      0.86       500

准确率: 0.854
